In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/simon-easy-ocr-data'
#role = 'arn:aws:iam::294038372338:role/hunkimSagemaker'

In [ ]:
#Make a dummy model.tar.gz
import tarfile
import io

with tarfile.open('model.tar.gz', "w:gz") as tar:
  dummy = b"dummy"
  info = tarfile.TarInfo('dummy.bin')
  info.size = len(dummy)
  tar.addfile(info, io.BytesIO(dummy))

In [ ]:
dummy_model = sagemaker_session.upload_data(path='model.tar.gz', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(dummy_model))

In [ ]:
from sagemaker.pytorch import PyTorchModel

# FIXME: Need to upload Dummy model (perhaps only model.targ.gz)
print(dummy_model)
model = PyTorchModel(model_data=dummy_model,
                     role=role,
                     framework_version='1.5.0',
                     py_version = "py3",
                     entry_point='ocr_infer.py',
                     source_dir='code')

In [ ]:
%%time
predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

In [ ]:
# Get the end point
endpoint = predictor.endpoint  
print(endpoint)
print("See the logs at", "https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/"+ endpoint)

In [ ]:
%%time
import boto3
import json

client = boto3.client('sagemaker-runtime')

img = open('code/example.jpg', 'rb').read()

response = client.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType='application/x-image', 
    #Accept="application/json" ,
    Body=bytearray(img)
)

print(response['Body'].read()) 

In [ ]:
sagemaker_session.delete_endpoint(predictor.endpoint)

In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()